In [ ]:
import pandas as pd
 
df = pd.read_csv('final_version.csv')
df

In [ ]:
## only gross
gross = df[df['commodity_transaction'] == 'Electricity - Gross production'].sort_values(['country_or_area', 'year'])
gross

In [ ]:
# only renewalbe or no-renewable
renewable_list = ['Electricity - total nuclear production',
                'Electricity - total geothermal production',
                'Electricity - total tide, wave and hydro production',
                'Electricity - total solar production',
                'Electricity - total wind production']

non_renewable_list = ['Electricity - total thermal production',
                    'From chemical heat – Autoproducer',
                    'From chemical heat – Main activity',
                    'From other sources – Autoproducer',
                    'From other sources – Main activity']

rene = df[(df['commodity_transaction'].isin(renewable_list))|(df['commodity_transaction'].isin(non_renewable_list))].sort_values(['country_or_area', 'year'])
rene

In [ ]:
def f_renewable(row):
    if row in renewable_list:
        return 1
    else:
        return 0

def f_non_renewable(row):
    if row in non_renewable_list:
        return 1
    else:
        return 0



In [ ]:
rene['renewable'] = rene['commodity_transaction'].apply(f_renewable)
rene['renewable'] = rene['new_quantity'] * rene['renewable']
rene['non_renewable'] = rene['commodity_transaction'].apply(f_non_renewable)
rene['non_renewable'] = rene['new_quantity'] * rene['non_renewable']
rene

In [ ]:
x = rene.groupby(['country_or_area', 'ISO', 'year']).agg({'renewable':'sum', 'non_renewable': 'sum'}).reset_index()
x

In [ ]:
y = pd.merge(gross, x, on = ['country_or_area','ISO','year'], how= 'left')
y

In [ ]:
#checking 
y['difference'] = (y['new_quantity'] - y['renewable'] - y['non_renewable']) == 0
y

In [ ]:
y['difference'].value_counts()

In [ ]:
y.to_csv('renewable.csv', index = False)